In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

In [3]:
from pyspark.sql.types import *

### func 

In [4]:
def list_to_set(source_lst):
    a = set()
    for i in source_lst:
        a = a | set(i)
    return list(a)   

In [5]:
set_from_list_udf = f.udf(list_to_set, ArrayType(StringType()))

In [6]:
date2 = '2021-07-04'

In [7]:
date1 = '2021-04-01'

In [8]:
input_table = 'MA_MMB_OFFER_NONTOP'
input_schema = 'SBX_TEAM_DIGITCAMP'

### dict 

In [9]:
%%time
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(SBX_TEAM_DIGITCAMP, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)").cache()

CPU times: user 2.12 ms, sys: 2.16 ms, total: 4.28 ms
Wall time: 3.68 s


In [10]:
prod_dct.show(3)

,ID,CRM_PRODUCT_ID,PRODUCT_CRM_NM,CRM_ACTIVE_FLG,PRODUCT_CD_KSB,PRODUCT_CD_RGS,PRODUCT_CD_MMB,PRODUCT_CD_ASUP,PRODUCT_SHORT_NM,PRODUCT_FULL_NM,PARTNER_NM,CHANNEL_SALE,CHANNEL_KKSB_ISKRA,CHANNEL_KKSB_EFS,CHANNEL_RGS_ISKRA,CHANNEL_ECO_EXPERT,CHANNEL_MMB_MS,CHANNEL_MMB_CKR,CHANNEL_MMB_MKK,CHANNEL_DZO,AGG_FORMULA_KSB,AGG_FORMULA_RGS,CNT_DAY_DEAL,PRODUCT_SUBGROUP,PRODUCT_GROUP,PRODUCT_TYPE,PRODUCT_PRIORITY
0,575,1-1AFS898R,2216/Гарантия,1,None,None,2216GARANTY,guarantee,2216/Гарантия,2216/Гарантия,Сбербанк,1,0,0,0,0,1,1,1,0,0,0,30.0,Банковская гарантия,Банковская гарантия,НКД,0
1,4606,1-2MEJXN3X,Поддержка Москвы,1,None,None,PODDERZKAMOSCOW,None,Поддержка Москвы,Поддержка Москвы,Сбербанк,0,0,0,0,0,0,0,0,0,0,0,30.0,Небанковские сервисы,Небанковские сервисы,Экосистема,0
2,48,1-DDP18CD,1С бухгалтерия,1,1032,1032,1C_DIRECT_BANK,1c_direct_bank,1С бухгалтерия,1С бухгалтерия,Сбербанк,1,1,1,0,0,0,0,0,0,1,0,30.0,Корус,Экосистема,НКД,0


In [11]:
prod_dct.count()

131



дата фабрики не новее, чем дата расчета АЛС

нонтоп - состояние на текущий день
приорити - рейтинги на предыдущем расчете, заселектить одну или два дня расчета (посмотреть) - не агрегироть (максимум среднее)

брать средний потенциал вместо суммы

ma_mmb_offer_toxic




сделать юнион источников нонтоп и приорити

### Nontop 

In [17]:
def create_nontop_df(input_schema, input_table, date):
    
    nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DTTM >= timestamp('{}'))"\
                 .format(input_schema, input_table, date)).cache()
    
    nontop_need_columns = nontop.select(f.to_date(f.col('CREATE_DTTM')).alias('CREATE_DATE'),
                                        f.col("INN"),
                                        f.col("PRODUCT_ID"),
                                        f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                         .alias("SOURCES_LV2_LIST_NONTOP"),
                                        f.col("COEF_PRODUCT_CD")\
                                         .alias("POTENTIAL_NONTOP")).cache()
    
    nontop_all_sources = nontop_need_columns.groupby(f.col("CREATE_DATE"),
                                                     f.col("INN"),
                                                     f.col("PRODUCT_ID")).agg(f.collect_set("SOURCES_LV2_LIST_NONTOP")\
                                                                               .alias("SOURCES_LV2_LIST_NONTOP"),
                                                                              f.avg("POTENTIAL_NONTOP")\
                                                                               .alias("POTENTIAL_NONTOP")).cache()
    
    nontop_before_end = nontop_all_sources.withColumn("SOURCES_LV2_LIST_NONTOP",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_NONTOP")))
    
    nontop_final = nontop_before_end.join(prod_dct,
                                          nontop_before_end.PRODUCT_ID == prod_dct.ID,
                                          how='inner')\
                                    .select(f.col("CREATE_DATE"),
                                            f.col("INN"),
                                            f.col("CRM_PRODUCT_ID").alias("CRM_PRODUCT_ID"),
                                            f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                            f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                            f.col("SOURCES_LV2_LIST_NONTOP"),
                                            f.col("POTENTIAL_NONTOP")
                                           ).cache()
        
    
    return nontop_final

In [18]:
%%time

nontop_prepr = create_nontop_df(input_schema, input_table, date1)

CPU times: user 13.7 ms, sys: 6.75 ms, total: 20.4 ms
Wall time: 271 ms


In [19]:
%%time
nontop_prepr.count()

CPU times: user 67.8 ms, sys: 42.2 ms, total: 110 ms
Wall time: 40.3 s


33296952

In [20]:
nontop_prepr.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,010100017999,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
1,2021-04-30,010100206570,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
2,2021-04-30,010100333289,1-PJ1ZWPY,Страхование,Страхование,[OTR2],5.320272e+02
3,2021-04-30,010100448410,1-3BGP3MY,Страхование,Страхование,[OTR2],8.837892e+02
4,2021-04-30,010100637706,1-1GVC,Кредитование,Кредитование,"[OTR2, ЦА-КРЕДМСХ1528-0421, SEASON]",8.822050e+09
5,2021-04-30,0101008493,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
6,2021-04-30,0101012940,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
7,2021-04-30,010101428705,1-115HHYO,Бизнес-карта,Бизнес-карта,[OTR2],3.016152e+03
8,2021-04-30,010101626376,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
9,2021-04-30,010101626376,1-1GVC,Кредитование,Кредитование,[OTR2],3.634922e+03


In [22]:
date1

'2021-04-30'

### Priority 

In [41]:
def create_priority_df(input_schema, input_table, date):
    
    priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('{}')"\
                    .format(input_schema, input_table, date)).cache()
    
    priority_need_columns = priority.select(f.to_date(f.col('load_dttm')).alias('CREATE_DATE'),
                                            f.col("INN"),
                                            f.col("CRM_PRODUCT_ID"),
                                            f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                             .alias("SOURCES_LV2_LIST_PRIORITY"),
                                            f.col("offer_priority")\
                                             .alias("POTENTIAL_PRIORITY")).cache()
    
    priority_all_sources = priority_need_columns.groupby(f.col("CREATE_DATE"),
                                                         f.col("INN"),
                                                         f.col("CRM_PRODUCT_ID")).agg(f.collect_set("SOURCES_LV2_LIST_PRIORITY")\
                                                                                      .alias("SOURCES_LV2_LIST_PRIORITY"),
                                                                                     f.avg("POTENTIAL_PRIORITY")\
                                                                                      .alias("POTENTIAL_PRIORITY")).cache()
                                                         
    priority_before_end = priority_all_sources.withColumn("SOURCES_LV2_LIST_PRIORITY",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_PRIORITY")))
    
    priority_final = priority_before_end.join(prod_dct,
                                              priority_before_end.CRM_PRODUCT_ID == prod_dct.CRM_PRODUCT_ID,
                                              how='inner')\
                                        .select(f.col("CREATE_DATE"),
                                                f.col("INN"),
                                                priority_before_end.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                                f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                                f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                                f.col("SOURCES_LV2_LIST_PRIORITY"),
                                                f.col("POTENTIAL_PRIORITY")
                                                ).cache()
    
    
        
    
    return priority_final

In [42]:
priority_prepr = create_priority_df(input_schema, "OFFER_PRIORITY", date1)

In [43]:
%%time
priority_prepr.count()

CPU times: user 40.6 ms, sys: 32.9 ms, total: 73.5 ms
Wall time: 4.63 s


1294125

In [44]:
priority_prepr.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY
0,2021-04-06,0105053264,1-HGDJ1PU,Экосистема,Корус,"[ЦА-КОРСПКСББОЛ-0420, ЦА-КОРСПК1-0221]",7.365011e+02
1,2021-04-06,010515470405,1-1GVC,Кредитование,Кредитование,"[T181, OTR2, SEASON]",6.730691e+09
2,2021-04-06,0216003205,1-EO0QXLI,Кредитование,Кредитование,[ЦА-КРЕДСМАРТ-0120],6.687822e+09
3,2021-04-06,0242009549,1-2UI5D8E1,Экосистема,Деловая среда,[ЦА-ФОРДСУБ-0321],9.186420e+11
4,2021-04-06,0263024788,1-2UI5D8E1,Экосистема,Деловая среда,[ЦА-ФОРДСУБ-0321],9.186420e+11
5,2021-04-06,026303659999,1-XNQFAFZ,Экосистема,Деловая среда,[OTR2],3.786300e+03
6,2021-04-06,032396792100,1-1GW0,Зарплатные проекты,Зарплатные проекты,[ЦА-ПРОД2ГИСММБ-0120],6.280826e+03
7,2021-04-06,0326046730,1-1GW0,Зарплатные проекты,Зарплатные проекты,[ЦА-ФОТММБ-0221],6.280826e+03
8,2021-04-06,032618563841,1-2UI5D8E1,Экосистема,Деловая среда,[ЦА-ФОРДСББ-0321],9.279212e+11
9,2021-04-06,0516012240,1-1GW0,Зарплатные проекты,Зарплатные проекты,[ЦА-ФОТММБ-0221],5.676855e+03


### toxic 

In [79]:
def create_toxic_df(input_schema, input_table, date):
    toxic = hive.sql("select * from {}.{} where inn is not Null and create_dttm >= timestamp('{}')"\
                     .format(input_schema, input_table, date)).cache()
    
    toxic_need_columns = toxic.select(f.to_date(f.col('create_dttm')).alias('CREATE_DATE'),
                                      f.col("INN"),
                                      f.col("product_id"),
                                      f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                       .alias("SOURCES_LV2_LIST_TOXIC")).cache()
    
    toxic_all_sources = toxic_need_columns.groupby(f.col("CREATE_DATE"),
                                                   f.col("INN"),
                                                   f.col("product_id")).agg(f.collect_set("SOURCES_LV2_LIST_TOXIC")\
                                                                             .alias("SOURCES_LV2_LIST_TOXIC")).cache()
    
    toxic_before_end = toxic_all_sources.withColumn("SOURCES_LV2_LIST_TOXIC",
                                                    set_from_list_udf(f.col("SOURCES_LV2_LIST_TOXIC")))
    
    toxic_final = toxic_before_end.join(prod_dct,
                                        toxic_before_end.product_id == prod_dct.ID,
                                        how='inner')\
                                  .select(f.col("CREATE_DATE"),
                                          f.col("INN"),
                                          prod_dct.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                          f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                          f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                          f.col("SOURCES_LV2_LIST_TOXIC")
                                         ).cache()
        
    
    return toxic_final

In [80]:
toxic_prepr = create_toxic_df(input_schema, "ma_mmb_offer_toxic", date1)

In [81]:
toxic_prepr.count()

31904724

In [83]:
toxic_prepr.show(3)

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_TOXIC
0,2021-04-20,010100067333,1-115HHYO,Бизнес-карта,Бизнес-карта,[ЦА-ДБКМЛ-0421]
1,2021-04-20,0101004555,1-ZYQP07F,Экосистема,СберЗдоровье,[OTR2]
2,2021-04-20,010103158691,1-2UI5D8E1,Экосистема,Деловая среда,[ЦА-ФОРДСЮЗБ-0321]


In [86]:
toxic_prepr.select("CREATE_DATE").distinct().show()

,CREATE_DATE
0,2021-04-21
1,2021-04-20


### Joiner 

In [45]:
def join_priority_nontop(priority_final, nontop_final):
    
    result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                                 .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                                nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(priority_final.INN),
                                                nontop_final.INN).otherwise(priority_final.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                                nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                                nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                                nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"))\
                                 .cache()
            
    
    return result_table

In [ ]:
toxic_prepr.join()

In [107]:
def join_priority_nontop_toxic(priority_final, nontop_final, toxic_final):
    
    result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                                 .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                                nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(priority_final.INN),
                                                nontop_final.INN).otherwise(priority_final.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                                nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                                nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                                nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"))\
                                 .cache()
            
    tmp_potential_rating = result_table.join(toxic_final,
                                             (result_table.INN == toxic_final.INN) &
                                             (result_table.CREATE_DATE == toxic_final.CREATE_DATE) &
                                             (result_table.CRM_PRODUCT_ID == toxic_final.CRM_PRODUCT_ID),
                                             how='full_outer')\
                                        .select(f.when(f.isnull(result_table.CREATE_DATE),
                                                       toxic_final.CREATE_DATE).otherwise(result_table.CREATE_DATE)\
                                                                                .alias("CREATE_DATE"),
                                                f.when(f.isnull(result_table.INN),
                                                       toxic_final.INN).otherwise(result_table.INN)\
                                                                        .alias("INN"),
                                                f.when(f.isnull(result_table.CRM_PRODUCT_ID),
                                                       toxic_final.CRM_PRODUCT_ID).otherwise(result_table.CRM_PRODUCT_ID)\
                                                                                   .alias("CRM_PRODUCT_ID"),
                                                f.when(f.isnull(result_table.PRODUCT_GROUP),
                                                       toxic_final.PRODUCT_GROUP).otherwise(result_table.PRODUCT_GROUP)\
                                                                                  .alias("PRODUCT_GROUP"),
                                                f.when(f.isnull(result_table.PRODUCT_SUBGROUP),
                                                       toxic_final.PRODUCT_SUBGROUP).otherwise(result_table.PRODUCT_SUBGROUP)\
                                                                                     .alias("PRODUCT_SUBGROUP"),
                                                f.col("SOURCES_LV2_LIST_NONTOP"),
                                                f.col("POTENTIAL_NONTOP"),
                                                f.col("SOURCES_LV2_LIST_PRIORITY"),
                                                f.col("POTENTIAL_PRIORITY"),
                                                f.col("SOURCES_LV2_LIST_TOXIC"))\
                                 .cache()         
    
    return tmp_potential_rating

In [88]:
toxic_final = toxic_prepr

In [84]:
result_table = join_priority_nontop(priority_prepr, nontop_prepr)

In [85]:
result_table.count()

34564165

In [87]:
result_table.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY
0,2021-04-30,010100195247,1-3BGP3MY,Страхование,Страхование,[OTR2],8.837892e+02,None,None
1,2021-04-30,010100358540,1-XNQFAFZ,Экосистема,Деловая среда,"[ЦА-ДСПЛАТФМЛ-0221, OTR2]",4.244400e+03,None,None
2,2021-04-30,010100400827,1-1AFS898R,Банковская гарантия,Банковская гарантия,[ЦА-ПРЕДОДОБАСТММБ-0620],8.588580e+03,None,None
3,2021-04-30,010100600960,1-1Z85A72,Кредитование,Кредитование,[OTR2],2.450188e+03,None,None
4,2021-04-30,0101007806,1-ZYQP07F,Экосистема,СберЗдоровье,[OTR2],6.408864e+02,None,None
5,2021-04-30,0101010132,1-MP71XI8,Экосистема,Юридические решения,[OTR2],4.118400e+02,None,None
6,2021-04-30,0101010492,1-1GVC,Кредитование,Кредитование,"[OTR2, SEASON]",8.822050e+09,None,None
7,2021-04-30,010101481427,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03,None,None
8,2021-04-30,010102372195,1-K3MF6Z8,Кредитование,Кредитная бизнес-карта,[ЦА-КРЕДСМАРТ-0120],1.810304e+04,None,None
9,2021-04-30,010102787908,1-1GVC,Кредитование,Кредитование,"[OTR2, SEASON]",7.779226e+09,None,None


In [96]:
tmp_potential_rating = result_table.join(toxic_final,
                  (result_table.INN == toxic_final.INN) &
                  (result_table.CREATE_DATE == toxic_final.CREATE_DATE) &
                  (result_table.CRM_PRODUCT_ID == toxic_final.CRM_PRODUCT_ID),
                  how='full_outer')\
            .select(f.when(f.isnull(result_table.CREATE_DATE),
                                                toxic_final.CREATE_DATE).otherwise(result_table.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(result_table.INN),
                                                toxic_final.INN).otherwise(result_table.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(result_table.CRM_PRODUCT_ID),
                                                toxic_final.CRM_PRODUCT_ID).otherwise(result_table.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(result_table.PRODUCT_GROUP),
                                                toxic_final.PRODUCT_GROUP).otherwise(result_table.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(result_table.PRODUCT_SUBGROUP),
                                                toxic_final.PRODUCT_SUBGROUP).otherwise(result_table.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"),
                                         f.col("SOURCES_LV2_LIST_TOXIC"))\
                                 .cache()

In [97]:
tmp_potential_rating.count()

66468722

In [112]:
tmp_potential_rating.filter("SOURCES_LV2_LIST_PRIORITY is not Null and SOURCES_LV2_LIST_NONTOP is not null").show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY,SOURCES_LV2_LIST_TOXIC
0,2021-04-30,0274912295,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03,[OTR2],3.928853e+03,None
1,2021-04-30,1419006563,1-16IN57Z0,Кредитование,Кредитование,[ЦА-КРЕДСМАРТТЕНДЕР-0120],1.842300e+06,[ЦА-КРЕДСМАРТТЕНДЕР-0120],1.842300e+06,None
2,2021-04-30,1645031309,1-115HHZC,РКО,РКО,"[ЦА-ПРИВЛКЛЮЧ-0321, ЦА-ПРИВЛКЛЮЧ-0720]",2.327287e+03,"[ЦА-ПРИВЛКЛЮЧ-0321, ЦА-ПРИВЛКЛЮЧ-0720]",2.327287e+03,None
3,2021-04-30,165914324103,1-1GUC,Эквайринг,Эквайринг,"[OTR2, ЦА-ЭКВММБПИР-1220]",5.543307e+03,"[OTR2, ЦА-ЭКВММБПИР-1220]",5.543307e+03,None
4,2021-04-30,183210138131,1-1GVC,Кредитование,Кредитование,"[OTR2, SEASON]",8.158585e+09,"[OTR2, SEASON]",8.158585e+09,None
5,2021-04-30,2309147002,1-EO0QXLI,Кредитование,Кредитование,[ЦА-КРЕДСМАРТ-0120],1.164607e+10,[ЦА-КРЕДСМАРТ-0120],1.164607e+10,None
6,2021-04-30,2309153158,1-EO0QXLI,Кредитование,Кредитование,[ЦА-КРЕДСМАРТ-0120],1.164607e+10,[ЦА-КРЕДСМАРТ-0120],1.164607e+10,None
7,2021-04-30,234802364678,1-115HHZC,РКО,РКО,[ЦА-ПРИВЛКЛЮЧ-0720],2.538227e+03,[ЦА-ПРИВЛКЛЮЧ-0720],2.538227e+03,None
8,2021-04-30,2508138108,1-115HHZC,РКО,РКО,[ЦА-ПРИВЛКЛЮЧ-0321],3.063484e+03,[ЦА-ПРИВЛКЛЮЧ-0321],3.063484e+03,None
9,2021-04-30,262804188039,1-115HHZC,РКО,РКО,"[ЦА-ПРИВЛКЛЮЧ-0321, ЦА-ПРИВЛКЛЮЧИНТНЛЗ-0321, ЦА-ПРИВЛКЛЮЧ-0720]",2.538226e+09,"[ЦА-ПРИВЛКЛЮЧ-0321, ЦА-ПРИВЛКЛЮЧИНТНЛЗ-0321, ЦА-ПРИВЛКЛЮЧ-0720]",2.538226e+09,None


In [108]:
tmp_final = join_priority_nontop_toxic(priority_prepr, nontop_prepr, toxic_prepr)

In [109]:
tmp_final.count()

66468722

In [110]:
create_table_from_df(SBX_TEAM_DIGITCAMP, "POTENTIAL_ALL_SOURCES_TMP", tmp_final, hive)

In [111]:
tmp_final.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY,SOURCES_LV2_LIST_TOXIC
0,2021-04-21,010100159390,1-1GVC,Кредитование,Кредитование,None,NaN,None,None,"[OTR2, SEASON]"
1,2021-04-30,010100195247,1-3BGP3MY,Страхование,Страхование,[OTR2],883.789185,None,None,None
2,2021-04-30,010100358540,1-XNQFAFZ,Экосистема,Деловая среда,"[ЦА-ДСПЛАТФМЛ-0221, OTR2]",4244.399902,None,None,None
3,2021-04-30,010100400827,1-1AFS898R,Банковская гарантия,Банковская гарантия,[ЦА-ПРЕДОДОБАСТММБ-0620],8588.580078,None,None,None
4,2021-04-20,0101004724,1-3BGP3MY,Страхование,Страхование,None,NaN,None,None,[OTR2]
5,2021-04-30,010100600960,1-1Z85A72,Кредитование,Кредитование,[OTR2],2450.188232,None,None,None
6,2021-04-21,0101006986,1-115HHYO,Бизнес-карта,Бизнес-карта,None,NaN,None,None,[OTR2]
7,2021-04-30,0101007806,1-ZYQP07F,Экосистема,СберЗдоровье,[OTR2],640.886414,None,None,None
8,2021-04-21,0101008736,1-415WQ1A,Страхование,Страхование,None,NaN,None,None,[ЦА-ЗСМ-0121]
9,2021-04-21,010100878451,1-XNQFAFZ,Экосистема,Деловая среда,None,NaN,None,None,[OTR2]
